## Quickstart

Examples using the Azure Function proxy to call LLMs in the private cloud.

1. Install Dependencies

In [ ]:
!pip install openai python-dotenv dspy

2. Create .env file

Create a file named .env in your project folder:

```
# Base URL of the proxy (must end with /api/v1)
PROXY_BASE_URL=https://<your-function-app>.azurewebsites.net/api/v1

# Shared host key (treat like a password)
FUNCTION_HOST_KEY=<paste-the-shared-host-key-here>

```

### Example: OpenAI Responses API

In [11]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

base_url = os.environ["PROXY_BASE_URL"].rstrip("/")
fn_key = os.environ["FUNCTION_HOST_KEY"]

client = OpenAI(
    base_url=base_url,
    api_key="unused",  # required by SDK; auth is via x-functions-key
    default_headers={"x-functions-key": fn_key},
)

resp = client.responses.create(
    model="gpt-4o-mini",
    input="Tell me a one line story.",
    temperature=0.3,
)

print(resp.output_text)

Beneath the faint glow of the moon, the lost letter finally found its way home, reuniting two souls who had always been meant to cross paths.


### Example: DSPy

In [15]:
import dspy
lm = dspy.LM(
    "openai/gpt-4o-mini",
    api_base=base_url,
    api_key="unused",  # required by OpenAI/LiteLLM; proxy ignores it
    extra_headers={"x-functions-key": fn_key},  # <-- the important part
)

dspy.configure(lm=lm)

pred = dspy.Predict("question -> answer")(question="Tell me a one line story.")
print(pred.answer)

As the clock struck midnight, she finally opened the letter that would change her life forever.
